In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd
import json, datetime

In [2]:
root_path = ''

with open(root_path+'appdata/db_auth.json') as f:
        temp = json.load(f)
        server = temp['server']
        database = temp['database'] 
        username = temp['username'] 
        password = temp['password']

connection_string = 'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)

In [3]:
query = f"""
SELECT 
    item.ItemLookupCode, 
    sp.SupplierID, 
    dbo.Supplier.SupplierName, 
    dbo.Supplier.Code, 
    item.Description as Description,
    item.Price Price, 
    item.Cost itemCost, 
    sp.Cost spCost
FROM 
    dbo.Item item
LEFT JOIN 
    dbo.SupplierList sp ON item.ID=sp.ItemID
LEFT JOIN 
    dbo.Supplier ON sp.SupplierID=dbo.Supplier.ID
WHERE 
    item.DepartmentID IN (2,4,6)
    AND item.SupplierID IN (2115, 2121)
    AND Item.Inactive = 0
ORDER BY 
    item.SupplierID,
    item.Description;
"""

with engine.connect() as conn, conn.begin():  
    fromPOS = pd.read_sql(query, conn, dtype={'ItemLookupCode': str})

In [4]:
fromPOS['Cost']= fromPOS[['itemCost', 'spCost']].apply(max, axis=1)
fromPOS


,ItemLookupCode,SupplierID,SupplierName,Code,Description,Price,itemCost,spCost,Cost
0,827298592051,2115,SUN TAIYANG CO LTD,10086,BBLB18 1B,79.99,5.0,5.0,5.0
1,827298625209,2115,SUN TAIYANG CO LTD,10086,BBLB18 27,79.99,5.0,5.0,5.0
2,827298625216,2115,SUN TAIYANG CO LTD,10086,BBLB18 30,79.99,5.0,5.0,5.0
3,827298625223,2115,SUN TAIYANG CO LTD,10086,BBLB18 33,79.99,5.0,5.0,5.0
4,827298592129,2115,SUN TAIYANG CO LTD,10086,BBLB18 F1B/33,79.99,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...
9124,803868474224,2121,SENSATIONNEL,10105,WIGGIE W104,19.99,9.0,9.0,9.0
9125,803868474231,2121,SENSATIONNEL,10105,WIGGIE W105,19.99,9.0,9.0,9.0
9126,803868474248,2121,SENSATIONNEL,10105,WIGGIE W106,19.99,9.0,9.0,9.0
9127,803868474255,2121,SENSATIONNEL,10105,WIGGIE W107,19.99,9.0,9.0,9.0


In [6]:
low_price = fromPOS[fromPOS['Price']<fromPOS['Cost']]

: 

In [42]:
fromPOS = fromPOS[['ItemLookupCode', 'SupplierName', 'Description', 'Cost']]

In [43]:
fromPOS

,ItemLookupCode,SupplierName,Description,Cost
0,827298592051,SUN TAIYANG CO LTD,BBLB18 1B,5.0
1,827298625209,SUN TAIYANG CO LTD,BBLB18 27,5.0
2,827298625216,SUN TAIYANG CO LTD,BBLB18 30,5.0
3,827298625223,SUN TAIYANG CO LTD,BBLB18 33,5.0
4,827298592129,SUN TAIYANG CO LTD,BBLB18 F1B/33,5.0
...,...,...,...,...
9095,803868474224,SENSATIONNEL,WIGGIE W104,9.0
9096,803868474231,SENSATIONNEL,WIGGIE W105,9.0
9097,803868474248,SENSATIONNEL,WIGGIE W106,9.0
9098,803868474255,SENSATIONNEL,WIGGIE W107,9.0


In [44]:
all_amazon = pd.read_csv(root_path+'inv_data/Amazon_All+Listings+Report.txt', sep='\t', dtype={'product-id': str})
all_amazon = all_amazon[['seller-sku', 'asin1', 'listing-id',
       'price', 'product-id-type', 'product-id', 'status']]

In [50]:
test = fromPOS.merge(all_amazon, how='left', left_on='ItemLookupCode', right_on='product-id')

In [48]:
fromPOS.to_excel('test.xlsx')

In [51]:
test

,ItemLookupCode,SupplierName,Description,Cost,seller-sku,asin1,listing-id,price,product-id-type,product-id,status
0,827298592051,SUN TAIYANG CO LTD,BBLB18 1B,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,827298625209,SUN TAIYANG CO LTD,BBLB18 27,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,827298625216,SUN TAIYANG CO LTD,BBLB18 30,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,827298625223,SUN TAIYANG CO LTD,BBLB18 33,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,827298592129,SUN TAIYANG CO LTD,BBLB18 F1B/33,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9171,803868474224,SENSATIONNEL,WIGGIE W104,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9172,803868474231,SENSATIONNEL,WIGGIE W105,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9173,803868474248,SENSATIONNEL,WIGGIE W106,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9174,803868474255,SENSATIONNEL,WIGGIE W107,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
test.dropna()

,ItemLookupCode,SupplierName,Description,Cost,seller-sku,asin1,listing-id,price,product-id-type,product-id,status
138,827298786726,SUN TAIYANG CO LTD,BDJOC5 1,11.50,OT_JAMAICAN OCEAN WAVE 5PCS_1,B07F3GG3DY,0701Z13MVAT,32.19,3.0,827298786726,Inactive
139,827298786733,SUN TAIYANG CO LTD,BDJOC5 1B,11.50,OT_JAMAICAN OCEAN WAVE 5PCS_1B,B07F3J5R1B,0701Z13HI8Q,32.19,3.0,827298786733,Inactive
140,827298786740,SUN TAIYANG CO LTD,BDJOC5 2,11.50,OT_JAMAICAN OCEAN WAVE 5PCS_2,B07F3NGFL3,0701Z15U638,32.19,3.0,827298786740,Inactive
141,827298789383,SUN TAIYANG CO LTD,BDJOC5 4,11.50,OT_JAMAICAN OCEAN WAVE 5PCS_4,B07F3R4G5G,0701Z15PSWF,32.19,3.0,827298789383,Inactive
142,827298786757,SUN TAIYANG CO LTD,BDJOC5 950,11.50,OT_JAMAICAN OCEAN WAVE 5PCS_950,B07F3GFX1W,0701Z12VK8M,31.95,3.0,827298786757,Active
...,...,...,...,...,...,...,...,...,...,...,...
9163,803868523410,SENSATIONNEL,VBMDE182022 BALAYAGEAUBURN,8.25,HZ_VBMDE182022_BALAYAGEAUBURN,B0B5VMPRH4,1121AAO7ZPL,34.49,3.0,803868523410,Active
9164,803868523427,SENSATIONNEL,VBMDE182022 BALAYAGEBLONDE,8.25,HZ_VBMDE182022_BALAYAGEBLONDE,B0B5VQL81N,1121AAO5XLE,34.49,3.0,803868523427,Inactive
9165,803868523021,SENSATIONNEL,VBMDE182022 T2/27,8.25,HZ_VBMDE182022_T2/27,B0B5VMTX27,1121AAQZYIN,34.49,3.0,803868523021,Active
9166,803868523038,SENSATIONNEL,VBMDE182022 T2/30,8.25,HZ_VBMDE182022_T2/30,B0B5VMJWPS,1121AAO5NFJ,34.49,3.0,803868523038,Active


In [58]:
dup = test[test.duplicated('ItemLookupCode')]

In [59]:
test.to_excel('test.xlsx', index=False)